In [ ]:
# importing some python modules - just in case
import pandas as pd
import numpy as np
import os, sys 
import io
import ee

In [ ]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt-get update

In [ ]:
#pyspark
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

In [ ]:
from pyspark import SparkConf                                                                                                                 
from pyspark.context import SparkContext                                                                                                      
from pyspark.sql import SparkSession, SQLContext

In [ ]:
# from pyspark.sql.session import SparkSession

# spark = SparkSession.builder.getOrCreate()

In [ ]:

from google.colab import files
drive.mount('/content/drive')



In [ ]:
# drive.mount("/content/drive", force_remount=True)

In [ ]:
path = "/content/drive/My Drive/data/clean_ppp_data.csv"

In [ ]:
# df=pd.read_csv(path)
# df.sample()

In [ ]:
# df.info()

In [ ]:
#need to change NAICS to string
# https://stackoverflow.com/questions/46956026/how-to-convert-column-with-string-type-to-int-form-in-pyspark-data-frame

In [ ]:
# url="https://drive.google.com/file/d/1k-atuiveaRBoDy02akkgmoawulTsXQH8/view?usp=sharing"

df=spark.read.csv(SparkFiles.get(path), sep=",", header=True)
df.show()

In [ ]:
df = df.withColumn("NAICS Code", df['NAICS Code'].cast('string'))

In [ ]:
df.printSchema()

In [ ]:
# df.describe()

In [ ]:
# notes... to do calc on existing and put in new example
#dataframe.withColumn('newname', dataframe['target']*2)

In [ ]:
gender=df.orderBy(df['Gender'].asc())
gender.show()

In [ ]:
from pyspark.sql.functions import col, avg
womens = df.filter(df['Gender']== 'Female Owned')
mens = df.filter(df['Gender']== 'Male Owned')



In [ ]:
womens.describe().show()


In [ ]:
mens.describe().show()



In [ ]:
df.select(avg("Loan Amount")).show()

In [ ]:
gen_vets=df.groupBy('Gender', 'Veteran').count().orderBy('count')
gen_vets.show()

In [ ]:
vet_ct=df.groupBy('Veteran').count().orderBy('count')
vet_ct.show()

In [ ]:
rc_ct=df.groupBy('Race').count().orderBy('count', ascending=False)
rc_ct.show()

In [ ]:
rc_ct=df.groupBy('Race', 'Veteran', 'Gender').count().orderBy('count', ascending=False)
rc_ct.show()

In [ ]:
gender_lns=df.filter("Loan Amount")
gender_lns.show()

In [ ]:
# group_by_dataframe.count().filter("`count` >= 10").orderBy('count', ascending=False)
# value counts for biz codes that occur more than 5000 times
from pyspark.sql.functions import col

(df
    .count()
    .filter("`count` >= 5000")
    .sort(col("count").desc()))
nct=df.count().filter("`Race` >= 5000").orderBy('count', ascending=False)
nct.show()



In [ ]:
df.to_csv('ppp_pyspark.csv', index=0)
!cp ppp_pyspark.csv "drive/My Drive/"


In [ ]:
# Load transformed raw data (the dataframes just created) into postgres database
# Configure settings for RDS
# mode = "append"
# jdbc_url="jdbc:postgresql://challenge.cyngxsnzir4o.us-east-2.rds.amazonaws.com:5432/postgres"
# config = {"user":"postgres", 
#           "password": "your-password-here", 
#           "driver":"org.postgresql.Driver"}